In [1]:
from pathlib import Path

import pandas as pd
import functions.data_acquisition as data_funcs


%cd ..

import src.configuration as config

Path(config.RAW_DATA_DIRECTORY).mkdir(parents=True, exist_ok=True)
Path(config.INTERIM_DATA_DIRECTORY).mkdir(parents=True, exist_ok=True)
Path(config.PROCESSED_DATA_DIRECTORY).mkdir(parents=True, exist_ok=True)

/workspaces/Tab_4_5_aviation_final_project


In [2]:
data_funcs.download_data(config.INCIDENT_DATA_URL, config.RAW_DATA_DIRECTORY, config.RAW_INCIDENTS_MDB_FILE)

In [3]:
table=data_funcs.parse_mdb(config.RAW_INCIDENTS_MDB_FILE, config.RAW_INCIDENTS_CSV_FILE)

In [12]:
import subprocess
import pandas as pd
import io
import os

mdb_file = "/workspaces/Tab_4_5_aviation_final_project/data/raw/avall.mdb"
output_dir = "/workspaces/Tab_4_5_aviation_final_project/data/processed"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Step 1: List tables in the MDB file
def list_tables(mdb_file):
    result = subprocess.run(["mdb-tables", "-1", mdb_file], stdout=subprocess.PIPE, text=True)
    tables = result.stdout.strip().split("\n")
    tables = [t.strip() for t in tables if t.strip()]
    print(f"Found tables: {tables}")
    return tables

# Step 2: Export and save each table to CSV
def load_and_save_tables(mdb_file, tables, output_dir):
    dataframes = {}
    for table in tables:
        print(f"Exporting table: {table}")
        result = subprocess.run(["mdb-export", mdb_file, table], stdout=subprocess.PIPE, text=True)
        csv_output = result.stdout
        if csv_output.strip():  # Check if there's any data
            try:
                df = pd.read_csv(io.StringIO(csv_output))
                dataframes[table] = df
                csv_path = os.path.join(output_dir, f"{table}.csv")
                df.to_csv(csv_path, index=False)
                print(f"Saved '{table}' to {csv_path} (shape={df.shape})")
            except Exception as e:
                print(f"Failed to read table {table}: {e}")
        else:
            print(f"Table {table} has no data.")
    return dataframes

# Run the full process
tables = list_tables(mdb_file)
dataframes = load_and_save_tables(mdb_file, tables, output_dir)

Found tables: ['Country', 'ct_iaids', 'ct_seqevt', 'dt_events', 'dt_Flight_Crew', 'eADMSPUB_DataDictionary', 'engines', 'events', 'Events_Sequence', 'Flight_Crew', 'flight_time', 'injury', 'NTSB_Admin', 'Occurrences', 'seq_of_events', 'states', 'aircraft', 'dt_aircraft', 'Findings', 'narratives']
Exporting table: Country
Saved 'Country' to /workspaces/Tab_4_5_aviation_final_project/data/processed/Country.csv (shape=(259, 2))
Exporting table: ct_iaids
Saved 'ct_iaids' to /workspaces/Tab_4_5_aviation_final_project/data/processed/ct_iaids.csv (shape=(955, 11))
Exporting table: ct_seqevt
Saved 'ct_seqevt' to /workspaces/Tab_4_5_aviation_final_project/data/processed/ct_seqevt.csv (shape=(2224, 2))
Exporting table: dt_events
Saved 'dt_events' to /workspaces/Tab_4_5_aviation_final_project/data/processed/dt_events.csv (shape=(104960, 5))
Exporting table: dt_Flight_Crew
Saved 'dt_Flight_Crew' to /workspaces/Tab_4_5_aviation_final_project/data/processed/dt_Flight_Crew.csv (shape=(166157, 7))
Exp

/tmp/ipykernel_17797/2645327537.py:29: DtypeWarning: Columns (0,10,31,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(csv_output))


Saved 'events' to /workspaces/Tab_4_5_aviation_final_project/data/processed/events.csv (shape=(28791, 73))
Exporting table: Events_Sequence
Saved 'Events_Sequence' to /workspaces/Tab_4_5_aviation_final_project/data/processed/Events_Sequence.csv (shape=(62030, 10))
Exporting table: Flight_Crew
Saved 'Flight_Crew' to /workspaces/Tab_4_5_aviation_final_project/data/processed/Flight_Crew.csv (shape=(30034, 33))
Exporting table: flight_time


/tmp/ipykernel_17797/2645327537.py:29: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(csv_output))


Saved 'flight_time' to /workspaces/Tab_4_5_aviation_final_project/data/processed/flight_time.csv (shape=(376601, 8))
Exporting table: injury


...

Saved 'injury' to /workspaces/Tab_4_5_aviation_final_project/data/processed/injury.csv (shape=(161761, 7))
Exporting table: NTSB_Admin
Saved 'NTSB_Admin' to /workspaces/Tab_4_5_aviation_final_project/data/processed/NTSB_Admin.csv (shape=(28792, 5))
Exporting table: Occurrences
Saved 'Occurrences' to /workspaces/Tab_4_5_aviation_final_project/data/processed/Occurrences.csv (shape=(0, 8))
Exporting table: seq_of_events
Saved 'seq_of_events' to /workspaces/Tab_4_5_aviation_final_project/data/processed/seq_of_events.csv (shape=(0, 11))
Exporting table: states
Saved 'states' to /workspaces/Tab_4_5_aviation_final_project/data/processed/states.csv (shape=(51, 3))
Exporting table: aircraft


/tmp/ipykernel_17797/2645327537.py:29: DtypeWarning: Columns (0,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(csv_output))


Saved 'aircraft' to /workspaces/Tab_4_5_aviation_final_project/data/processed/aircraft.csv (shape=(29264, 93))
Exporting table: dt_aircraft
Saved 'dt_aircraft' to /workspaces/Tab_4_5_aviation_final_project/data/processed/dt_aircraft.csv (shape=(247661, 6))
Exporting table: Findings


/tmp/ipykernel_17797/2645327537.py:29: DtypeWarning: Columns (0,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(csv_output))


Saved 'Findings' to /workspaces/Tab_4_5_aviation_final_project/data/processed/Findings.csv (shape=(68003, 14))
Exporting table: narratives
Saved 'narratives' to /workspaces/Tab_4_5_aviation_final_project/data/processed/narratives.csv (shape=(26834, 8))


In [17]:
import pandas as pd
import os

# Set directory and files to join
input_dir = "/workspaces/Tab_4_5_aviation_final_project/data/processed"
output_file = "/workspaces/Tab_4_5_aviation_final_project/data/processed/combined_output.csv"

# Define CSV files and the common column to join on
csv_files = [
    ("events", "ev_id"),
    ("aircraft", "ev_id"),
    ("injury", "ev_id")
]
# ^ These are table names without the .csv extension; must exist in input_dir

# Load the first file as the base DataFrame
base_name, join_key = csv_files[0]
base_df = pd.read_csv(os.path.join(input_dir, f"{base_name}.csv"))

# Iteratively join with the other tables
for table_name, join_col in csv_files[1:]:
    df_to_join = pd.read_csv(os.path.join(input_dir, f"{table_name}.csv"))
    print(f"Joining {table_name} on {join_col}")
    base_df = pd.merge(base_df, df_to_join, on=join_col, how="left")  # or "inner", "outer", etc.

# Save combined DataFrame
base_df.to_csv(output_file, index=False)
print(f"Combined DataFrame saved to: {output_file}")

/tmp/ipykernel_17797/1866099444.py:18: DtypeWarning: Columns (0,10,31,51) have mixed types. Specify dtype option on import or set low_memory=False.
  base_df = pd.read_csv(os.path.join(input_dir, f"{base_name}.csv"))
/tmp/ipykernel_17797/1866099444.py:22: DtypeWarning: Columns (0,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df_to_join = pd.read_csv(os.path.join(input_dir, f"{table_name}.csv"))


Joining aircraft on ev_id
Joining injury on ev_id


/tmp/ipykernel_17797/1866099444.py:22: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_to_join = pd.read_csv(os.path.join(input_dir, f"{table_name}.csv"))


Combined DataFrame saved to: /workspaces/Tab_4_5_aviation_final_project/data/processed/combined_output.csv
